In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from joblib import dump

from my_krml_25176165.data.sets import load_sets

X_train, y_train, X_val, y_val, X_test, y_test = load_sets()

X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


((30000, 8), (30000,), (10000, 8), (10000,), (10000, 8), (10000,))

In [ ]:
# Apply Polynomial Transformation

In [3]:
from sklearn.preprocessing import PolynomialFeatures

# [3.2] 建立 PolynomialFeatures（degree=2）
poly = PolynomialFeatures(degree=2)

# [3.3] 對 X_train 擬合並轉換
X_train = poly.fit_transform(X_train)

# [3.4] 查看轉換後的維度
X_train.shape

# [3.5] 對 X_val 與 X_test 進行相同轉換（不重新 fit）
X_val = poly.transform(X_val)
X_test = poly.transform(X_test)


In [ ]:
# Train Linear Regression model

In [7]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()

reg.fit(X_train, y_train)

from pathlib import Path

Path("../models").mkdir(parents=True, exist_ok=True)
dump(reg, '../models/linear_poly_2.joblib')

y_train_preds = reg.predict(X_train)
y_val_preds = reg.predict(X_val)

from my_krml_25176165.models.performance import print_regressor_scores

print_regressor_scores(y_preds=y_train_preds, y_actuals=y_train, set_name='Training')

print_regressor_scores(y_preds=y_val_preds, y_actuals=y_val, set_name='Validation')

y_test_preds = reg.predict(X_test)
print_regressor_scores(y_preds=y_test_preds, y_actuals=y_test, set_name='Testing')



RMSE Training: 6107.132246057445
MAE Training: 4110.85016821403
RMSE Validation: 6124.952553912255
MAE Validation: 4130.709044283882
RMSE Testing: 6193.338727890162
MAE Testing: 4229.675154268106
